<a href="https://colab.research.google.com/github/armandoordonez/AI-Engineering/blob/main/Agentes_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from getpass import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass("Pega tu Google API key (Gemini): ")
os.environ["LANGCHAIN_API_KEY"] = getpass("Pega tu LangSmith API key: ")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "colab-agent"

Pega tu Google API key (Gemini): ··········
Pega tu LangSmith API key: ··········


In [ ]:
!pip install -U langchain langchain_google_genai langchain_community langgraph langsmith langgraph-checkpoint-sqlite python-dotenv wikipedia youtube_search

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.5 MB/s eta 0:00:00
  Created w

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_community.agent_toolkits.load_tools import get_all_tool_names
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
import os
from langchain.callbacks.tracers import LangChainTracer
from langsmith import traceable
from langchain.callbacks.manager import CallbackManager

In [ ]:
# Lee la clave desde la variable de entorno (usa getpass() antes)
g_api_key = os.getenv("GOOGLE_API_KEY")

# Inicialización
chat_model = ChatGoogleGenerativeAI(api_key=g_api_key, model="gemini-2.5-flash")

# Prueba rápida (mensaje simple)
resp = chat_model.invoke([HumanMessage(content="Resume 'Cien años de soledad' en 2 frases.")])
print(resp.content)

La novela narra la saga épica y fantástica de siete generaciones de la familia Buendía en el mítico pueblo de Macondo, donde la realidad se mezcla con lo mágico. A través de guerras, amores y tragedias, sus vidas están marcadas por la repetición cíclica y una soledad ineludible que los condena a la extinción final.


In [ ]:
messages = [SystemMessage(content='Eres un pirata gruñon'),
           HumanMessage(content="Resume 'Cien años de soledad' en 2 frases.")]

resp = chat_model.invoke(messages)
print(resp.content)

¡Arr! Es la maldita saga de la familia Buendía, que funda el pueblo de Macondo y vive un siglo de amores locos, guerras y tragedias, condenados a repetir sus errores por una profecía. Al final, todo se desvanece, el pueblo y la estirpe, consumidos por la soledad y el tiempo, ¡una maldición, digo yo!


In [ ]:
pelicula = "up"
template = "Cuentame sobre {pelicula}."
prompt = PromptTemplate(input_variables=["pelicula"], template=template)

chain = prompt | chat_model
out = chain.invoke({"pelicula": pelicula})
print(out)

content='¡Ah, "Up"! Es una de las películas animadas más queridas, emotivas y aclamadas de **Pixar Animation Studios**, producida por Walt Disney Pictures. Fue estrenada en 2009 y dirigida por Pete Docter.\n\nAquí te cuento sobre ella:\n\n1.  **La Trama Principal:**\n    La película sigue a **Carl Fredricksen**, un vendedor de globos jubilado de 78 años, quien es un viudo gruñón y solitario. Su vida ha estado marcada por el amor de su difunta esposa, **Ellie**, con quien compartió el sueño de toda la vida de viajar a las míticas **Cataratas Paraíso** (Paradise Falls) en Sudamérica, un lugar inspirado en los tepuyes de Venezuela.\n    Cuando Carl está a punto de ser desalojado de la casa que compartió con Ellie (y que tiene un inmenso valor sentimental), decide cumplir la promesa de su vida: ata miles de globos de helio a su casa, la arranca de sus cimientos y emprende el viaje hacia Sudamérica volando.\n\n2.  **El Inesperado Polizón:**\n    Lo que Carl no sabe es que tiene un polizón a

In [ ]:
get_all_tool_names
get_all_tool_names()[:10]

['sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-books',
 'google-lens']

In [ ]:
wiki_wrap = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrap, description="Resumen enciclopédico")

yt_tool = YouTubeSearchTool(description="Busca videos relevantes")

tools = [wiki_tool, yt_tool]

In [ ]:
model_with_tools = chat_model.bind_tools(tools)

# Pregunta general
print("Respuesta 1")
res1 = model_with_tools.invoke([HumanMessage(content="Hola")])
print("Texto:", res1.content)
print("Tool calls sugeridas:", res1.tool_calls)

#Pregunta específica
print("\nRespuesta 2")
res2 = model_with_tools.invoke([
   HumanMessage("Busca la biografía de Simon Bolivar")
])
print("Texto:", res2.content)
print("Tool calls sugeridas:", res2.tool_calls)

Respuesta 1
Texto: Hola, ¿en qué puedo ayudarte?
Tool calls sugeridas: []

Respuesta 2
Texto: 
Tool calls sugeridas: [{'name': 'wikipedia', 'args': {'query': 'Simon Bolivar'}, 'id': 'bd64d59a-9dac-443e-84fa-e78f95e8b3d8', 'type': 'tool_call'}]


In [ ]:
system_prompt = SystemMessage("Eres un asistente que elige entre texto, wiki o video según convenga.")

agent = create_react_agent(chat_model, tools, prompt=system_prompt)

# Función ejecutora básica
def execute(agent, query):
    response = agent.invoke({"messages": [HumanMessage(query)]})
    for m in response["messages"]:
        print(f"{m.__class__.__name__}: {m.content}\n{'-'*20}")
    return response

# Ejemplo
resp = execute(agent, "Explícame la serie de Fourier")

HumanMessage: Explícame la serie de Fourier
--------------------
AIMessage: 
--------------------
ToolMessage: Page: Fourier series
Summary: A Fourier series () is an expansion of a periodic function into a sum of trigonometric functions. The Fourier series is an example of a trigonometric series. By expressing a function as a sum of sines and cosines, many problems involving the function become easier to an
--------------------
AIMessage: La serie de Fourier es una forma de representar una función periódica como una suma infinita de funciones trigonométricas (senos y cosenos). Esto facilita el análisis y la resolución de problemas relacionados con esa función.
--------------------


In [ ]:
execute(agent, "qué recuerdas de mi anterior pregunta?")

HumanMessage: qué recuerdas de mi anterior pregunta?
--------------------
AIMessage: Lo siento, no tengo memoria de preguntas anteriores. Cada interacción es independiente.
--------------------


{'messages': [HumanMessage(content='qué recuerdas de mi anterior pregunta?', additional_kwargs={}, response_metadata={}, id='398a2c0a-d0f5-48f8-9b61-c5a8dbba5d5a'),
  AIMessage(content='Lo siento, no tengo memoria de preguntas anteriores. Cada interacción es independiente.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2afff31b-9c85-4674-8fe7-2657914d480c-0', usage_metadata={'input_tokens': 102, 'output_tokens': 52, 'total_tokens': 154, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 37}})]}

In [ ]:
checkpointer = MemorySaver()
agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)

def execute(agent, query, thread_id="a1b2c3"):
    config = {"configurable": {"thread_id": thread_id}}
    response = agent.invoke({"messages": [HumanMessage(query)]}, config=config)
    return response



print("\n Respuesta inicial")
resp1 = execute(agent, "Explícame la serie de Fourier", thread_id="a1b2c3")
print(resp1["messages"][-1].content)



print("\n Respuesta con memoria:")
resp2 = execute(agent, "¿Qué te he preguntado?", thread_id="a1b2c3")
print(resp2["messages"][-1].content)


 Respuesta inicial
La serie de Fourier es una forma de representar una función periódica como una suma infinita de funciones trigonométricas (senos y cosenos). Esto facilita el análisis de muchas funciones al transformarlas en una forma más manejable.

 Respuesta con memoria:
Me has preguntado sobre la serie de Fourier.


In [ ]:
resp1 = execute(agent, "Recuerda que mi color favorito es azul.", thread_id="demo123")
print(resp1["messages"][-1].content)

resp2 = execute(agent, "¿Cuál es mi color favorito?", thread_id="demo123")
print(resp2["messages"][-1].content)

# Forzar olvido: nuevo thread_id
resp3 = execute(agent, "¿Cuál es mi color favorito?", thread_id="otro123")
print(resp3["messages"][-1].content)

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 14.261684421s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 14
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 12.181337568s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flas

De acuerdo, recordaré que tu color favorito es azul.
Tu color favorito es azul.
Lo siento, no tengo acceso a tu información personal, por lo que no sé cuál es tu color favorito.


In [ ]:
tracer = LangChainTracer(project_name="colab-agent")
cb_manager = CallbackManager([tracer])

# Decorador traceable para marcar la función de ejecución
@traceable(name="execute_agent")
def execute(agent, query, thread_id="a1b2c3"):
    config = {"configurable":{"thread_id": thread_id}, "callbacks":[tracer]}
    response = agent.invoke({"messages":[HumanMessage(query)]}, config=config)
    return response

# Ejecuta y luego entra a LangSmith para ver trace.
resp = execute(agent, "Explícame las series de Fourier")


print(resp["messages"][-1].content)


Una serie de Fourier es una expansión de una función periódica en una suma de funciones trigonométricas. Es decir, te permite descomponer una función periódica compleja en una serie infinita de senos y cosenos más simples. Al hacer esto, muchos problemas que involucran la función se vuelven más fáciles de analizar.
